# Packages and input data

In [1]:
import os, re, csv
import subprocess
import numpy as np
import pandas as pd
import librosa
import soundfile
import sounddevice as sd
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import ccpreprocess, ccdownload, ccwebcrawl, ccchord, ccplay

np.random.seed(0)
torch.manual_seed(0)

dir_path   = os.path.abspath('')
excel_path = os.path.join(dir_path, 'ccClub music data (popular music).xlsx')
excel_data = pd.read_excel(excel_path, dtype={'Capo':int, '3 or 4':int})

excel_data = excel_data.iloc[:5]
# excel_data.head(5)

# Download and cut down the original music

In [4]:
cutdown_path = os.path.join(dir_path, 'music_cutdown')
if os.path.isdir(cutdown_path) == False:
    os.mkdir(cutdown_path)

for i in range(len(excel_data)):
    ccdownload.yt_wav(link=excel_data['Link'][i], file_name=excel_data['Title'][i])
    original_music = os.path.join(ccdownload.music_path, f"{excel_data['Title'][i]}.wav")
    cutdown_music  = os.path.join(cutdown_path, f"{excel_data['Title'][i]}.wav")
    print(f"Now cutting down {excel_data['Title'][i]} ... ", end='')
    if os.path.isfile(cutdown_music) == False:
        y, sr = librosa.load(original_music)
        cutdown_start = int(sr * excel_data['Start_Second'][i])
        cutdown_len   = int(len(excel_data['Chords'][i].strip('|').split('|')) * excel_data['3 or 4'][i] * (60/excel_data['Tempo'][i]) * sr)
        y = y[cutdown_start : cutdown_start + cutdown_len]
        soundfile.write(cutdown_music, y, sr)
        print(f"Cutting down finished!")
    else:
        print(f"File already cut!")

Now processing 心願便利貼 ... File already exists!
Now cutting down 心願便利貼 ... File already cut!
Now processing 飛鳥和蟬 ... File already exists!
Now cutting down 飛鳥和蟬 ... File already cut!
Now processing 晴天 ... File already exists!
Now cutting down 晴天 ... File already cut!
Now processing 不是因為天氣晴朗才愛你 ... File already exists!
Now cutting down 不是因為天氣晴朗才愛你 ... File already cut!
Now processing 披星戴月的想你 ... File already exists!
Now cutting down 披星戴月的想你 ... File already cut!


# Transform the input data

In [5]:
Musics = []
Chords = []

for i in range(len(excel_data)):
    padding_chord_list = ccchord.chord_padding(excel_data['Chords'][i])
    for c in range(len(padding_chord_list)):
        padding_chord_list[c] = ccchord.chord_simplify(padding_chord_list[c])
        padding_chord_list[c] = ccchord.chord_numeralize(padding_chord_list[c])
    Chords.append(np.array(padding_chord_list))

    y, sr = librosa.load(os.path.join(cutdown_path, f"{excel_data['Title'][i]}.wav"))
    Musics.append(ccpreprocess.Music(excel_data['Title'][i], y, sr, excel_data['Tempo'][i], sections=8,
                                    beats_per_section=4, slices_per_beat=excel_data['3 or 4'][i],
                                    f_min=-24, f_max=12, A4=440))
    

Now extracting amplitude of 心願便利貼 : -------------------------------------------------- Extraction  complete!
Now extracting amplitude of 飛鳥和蟬 : -------------------------------------------------- Extraction  complete!
Now extracting amplitude of 晴天 : -------------------------------------------------- Extraction  complete!
Now extracting amplitude of 不是因為天氣晴朗才愛你 : -------------------------------------------------- Extraction  complete!
Now extracting amplitude of 披星戴月的想你 : -------------------------------------------------- Extraction  complete!
